In [10]:
%load_ext autoreload
%autoreload 2

import sys
import qgrid
import pandas as pd
from pymongo import MongoClient

if ".." not in sys.path:
    sys.path.append("..")
    
from datafc.eval.report import DatasetReport

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
client = MongoClient()

datasets = ["nyc", "prog", "sygus"]

dataset_to_report = {}

for dataset in datasets:
    dataset_result = client.sacred.runs.find_one(
        {"config.dataset": dataset, "result": {"$ne": None}}, sort=[("$natural", -1)]
    )
    dataset_to_report[dataset] = DatasetReport(dataset, dataset_result["result"])

In [12]:
from ipywidgets import widgets
from ipywidgets import interact, interact_manual

dataset_select = widgets.Dropdown(description="dataset", options=datasets)
scenario_select = widgets.Dropdown(
    description="scenario",
    options=dataset_to_report[dataset_select.value].scenario_to_report.keys(),
)
category_select = widgets.Dropdown(
    description="category", options=["transformation", "validation"]
)


def show_scenarios(change):
    dataset = change["new"]
    scenario_select.options = dataset_to_report[dataset].scenario_to_report.keys()


dataset_select.observe(show_scenarios, names='value')


def show_failed_cases(dataset, scenario, category):
    if category == "transformation":
        failed_cases = dataset_to_report[dataset].scenario_to_report[scenario].failed_transformations
    else:
        failed_cases = dataset_to_report[dataset].scenario_to_report[scenario].failed_validations
    examples = [
        x
        for x in failed_cases
    ]
    df = pd.DataFrame(
        examples,
        columns=[
            "original_value",
            "transformed_values",
            "groundtruth_value",
            "validation_components",
            "validation_result",
        ],
    )
    display(df)


_ = interact(
    show_failed_cases,
    dataset=dataset_select,
    scenario=scenario_select,
    category=category_select,
)

interactive(children=(Dropdown(description='dataset', options=('nyc', 'prog', 'sygus'), value='nyc'), Dropdown…